In [ ]:
#default_exp dataflow

In [ ]:
#export
from nifi_api.endpoints import *
from nifi_api.rest import Processor,Flowfile
from nifi_api.tools import is_valid_csv

# Dataflow

> Monitors Source To Raw Process

In [ ]:
SOURCE_TO_RAW_IDS

{'S3ListSuccessConnection': {'Id': 'a9bc3f88-5f2d-1559-9c2d-625d8a66e5c8',
  'description': "S3 Bucket Check Files Processor's Success Relationship"},
 'RouteOnFilenameProcessor': {'Id': '1ac4364b-5f00-16b3-aee1-e15d8ad527ff',
  'description': 'Processor that filter an incoming list of CSVs'}}

In [ ]:
class SourceToRaw:
    
    first_connection = SOURCE_TO_RAW_IDS['S3ListSuccessConnection']['Id']
    first_processor = SOURCE_TO_RAW_IDS['RouteOnFilenameProcessor']['Id']

    @classmethod
    def start(cls):
        pass
    
    
    @classmethod
    def stop(cls):
        pass
    

In [ ]:
connection = SOURCE_TO_RAW_IDS['S3ListSuccessConnection']['Id']
Flowfile.list_queues(connection)

({'listingRequest': {'id': 'b7d8c13c-0177-1000-ffff-ffffe17f8608',
   'uri': 'https://learn-cluster-master0.sibatel.xo7z-6zib.cloudera.site:443/learn-cluster/cdp-proxy-api/nifi-app/nifi-api/flowfile-queues/a9bc3f88-5f2d-1559-9c2d-625d8a66e5c8/listing-requests/b7d8c13c-0177-1000-ffff-ffffe17f8608',
   'submissionTime': '02/19/2021 01:12:51.821 UTC',
   'lastUpdated': '01:12:51 UTC',
   'percentCompleted': 1,
   'finished': True,
   'maxResults': 100,
   'state': 'Completed successfully',
   'queueSize': {'byteCount': 0, 'objectCount': 3},
   'flowFileSummaries': [{'uri': 'https://learn-cluster-master0.sibatel.xo7z-6zib.cloudera.site:443/learn-cluster/cdp-proxy-api/nifi-app/nifi-api/flowfile-queues/a9bc3f88-5f2d-1559-9c2d-625d8a66e5c8/flowfiles/fba98d4f-a739-4952-9b96-120ce2b3725c',
     'uuid': 'fba98d4f-a739-4952-9b96-120ce2b3725c',
     'filename': 'cdp-data/resources/enrique/source/',
     'position': 1,
     'size': 0,
     'queuedDuration': 172129888,
     'lineageDuration': 172129

In [ ]:
class DataFlow:
    
    def __init__(self) -> None:
        self.init_connector: str
        self.init_processor: str
        self.final_connector: str
        self.final_processor: str
    #def __init__(self,init_connector,init_processor,final_connector,final_processor):
    #    self.init_connector: =
    
    
    @classmethod
    def switch_status(cls,status="RUNNING"):
        pass
    
    @classmethod
    def count_flowfiles(cls,connection_id):
        list_queues = Flowfile.list_queues(connection_id)

        pass
        


In [ ]:
'flowFileSummaries'

'flowFileSummaries'